In [4]:
import requests

Importation des données objets perdus (Lost Item)

In [5]:
def specific_url_lostitem(annee,mois):
    URL = "https://ressources.data.sncf.com/api/records/1.0/search/"
    ressource = "?dataset=objets-trouves-restitution&q=&rows=10000&sort=date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c&refine.date="
    month_format = f"{annee}%2F{mois}"
    return URL + ressource + month_format

In [6]:
from create_model import LostItem
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

# on crée un moteur de base de données (objet qui gère la connexion à la base de données et permet d'exécuter des requêtes SQL sur cette base de données.)
engine = create_engine('sqlite:///db.sqlite')

# on crée un "fabricant de session" pour créer une session de base de données qui gère l'ajout et la modification d'objets dans la BDD.)
Session = sessionmaker(bind=engine)

# on instancie une session de base de données
session = Session()

liste_mois = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
liste_annee = ["2016", "2017", "2018", "2019", "2020", "2021"]

# dictionnaire qui permet d'associer un nom de variable à une clé dans la boucle "for cle in dico.items()"
dico = {'date': 'date_var', 'gc_obo_type_c': 'type_var', 'gc_obo_gare_origine_r_name': 'gare_var', 'gc_obo_gare_origine_r_code_uic_c': 'code_uic_var', 
'gc_obo_date_heure_restitution_c': 'date_rest_var'}

for annee  in liste_annee:
    for mois in liste_mois:
        url = specific_url_lostitem(annee,mois)
        response = requests.get(url)
        
        for itemlost in response.json()["records"]:

            # on teste à chaque fois s'il existe une valeur que l'on peut enregistrer dans la base de données, sinon affecte la valeur None pour indiquer dans la BDD l'absence d'informations':
            for cle in dico.keys():
                try:
                    dico[cle] = itemlost['fields'][cle]
                except KeyError:
                    dico[cle] = None

            # on ajoute une nouvelle ligne dans la base de données avec les informations contenues dans le json
            session.add(LostItem(id = itemlost['recordid'],
                                date = dico['date'],
                                type_objet = dico['gc_obo_type_c'],
                                gare = dico['gc_obo_gare_origine_r_name'],
                                code_uic = dico['gc_obo_gare_origine_r_code_uic_c'],
                                date_restitution = dico['gc_obo_date_heure_restitution_c']))

# on  valide tous les changements apportés aux objets de la session et on les enregistre dans la base de données.
session.commit() 

Importation des données sur les gares

In [7]:
from create_model import Gare

Session = sessionmaker(bind=engine)
session = Session()

url = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=referentiel-gares-voyageurs&q=&rows=10000"

response = requests.get(url)

# liste dans laquelle on a regroupé par paire le nom d'une colonne de la table Gare avec le libellé correspondant dans le json
field_list = [
    ["code_uic", "uic_code"],
    ["nom_gare", "gare_alias_libelle_noncontraint"],
    ["code_commune", "commune_code"],
    ["code_postal", "adresse_cp"],
    ["code_departement", "departement_numero"],
    ["departement", "departement_libellemin"],
    ["longitude", "longitude_entreeprincipale_wgs84"],
    ["latitude", "latitude_entreeprincipale_wgs84"]
]

# pour chaque gare dans le fichier json
for gare in response.json()["records"]:
    
    # on initialise un dictionnaire qui contiendra les données que l'on veut récupérer concernant la gare
    gare_data= {}

    # on récupère l'id 
    gare_data["id"] = gare["recordid"]

    # pour chaque colonne
    for field in field_list:

        # on vérifie si la donnée existe pour la stocker dans la BDD, sinon on affecte la valeur none 
        try: 
            gare_data[field[0]] = gare["fields"][field[1]]
        except KeyError:
            gare_data[field[0]]=None

    # la syntaxe ** est utlisé  pour passer les valeurs du dico gare_data au constructeur de la classe Gare sans devoir réécrire les attributs de Gare
    session.add(Gare(**gare_data))
    session.commit()

Importation des données sur la fréquentation

In [8]:
from create_model import Frequentation

Session = sessionmaker(bind=engine)
session = Session()

url = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=frequentation-gares&q=&sort=nom_gare&rows=10000"
response = requests.get(url)

# liste dans laquelle on a regroupé par paire le nom d'une colonne de la table Frequentation avec le libellé correspondant dans le json
field_list = [
    ["code_uic", "code_uic_complet"],
    ["code_postal", "code_postal"],
    ["total_voyageurs_2016", "total_voyageurs_2016"],
    ["total_voyageurs_2017", "totalvoyageurs2017"],
    ["total_voyageurs_2018", "total_voyageurs_2018"],
    ["total_voyageurs_2019", "total_voyageurs_2019"],
    ["total_voyageurs_2020", "total_voyageurs_2020"],
    ["total_voyageurs_2021", "total_voyageurs_2021"]
]

for frequentation in response.json()["records"]:

    frequentation_data= {}
    frequentation_data["id"] = frequentation["recordid"]

    for field in field_list:
        try: 
            frequentation_data[field[0]] = frequentation["fields"][field[1]]
        except KeyError:
            frequentation_data[field[0]]=None

   
    session.add(Frequentation(**frequentation_data))
    session.commit()

Importation des données sur la température

In [12]:
from create_model import Temperature
import csv

Session = sessionmaker(bind=engine)
session = Session()

# ouvre le fichier CSV nommé "agregated_temperature.csv" en mode lecture ("r")
with open('agregated_temperature.csv', 'r') as csv_file:

    # l'objet csv.DictReader est un lecteur de fichiers CSV qui permet de lire chaque ligne du fichier CSV en tant que dictionnaire, avec les en-têtes de colonne comme clés et les valeurs de chaque colonne comme valeurs.
    csv_reader = csv.DictReader(csv_file)
    
    for row in csv_reader:
        temperature = Temperature(departement_code = row['departement_code'],
                                  date = row['Date'],
                                  temperature = row['temperature'])
        session.add(temperature)

session.commit()
